In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn import over_sampling, under_sampling
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
sample_submission = pd.read_csv('sample_submission.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train_new = train.drop(['Id'], axis=1)
test_new = test.drop(['Id'], axis=1)

X = ['age', 'years_of_experience', 'lesson_price', 'qualification',
       'physics', 'chemistry', 'biology', 'english', 'geography', 'history',
       'mean_exam_points']
Y = ['choose']

In [3]:
# # Дерево решений 0,74


# # Реализуем класс узла

# class Node:
    
#     def __init__(self, index, t, true_branch, false_branch):
#         self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
#         self.t = t  # значение порога
#         self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
#         self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

# # И класс терминального узла (листа)

# class Leaf:
    
#     def __init__(self, data, labels):
#         self.data = data
#         self.labels = labels
#         self.prediction = self.predict()
        
#     def predict(self):
#         # подсчет количества объектов разных классов
#         classes = {}  # сформируем словарь "класс: количество объектов"
#         for label in self.labels:
#             if label not in classes:
#                 classes[label] = 0
#             classes[label] += 1
            
#         # найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
#         prediction = max(classes, key=classes.get)
#         return prediction        

# # Расчет критерия Джини

# def gini(labels):
#     #  подсчет количества объектов разных классов
#     classes = {}
#     for label in labels:
#         if label not in classes:
#             classes[label] = 0
#         classes[label] += 1
    
#     #  расчет критерия
#     impurity = 1
#     for label in classes:
#         p = classes[label] / len(labels)
#         impurity -= p ** 2
        
#     return impurity

# # Расчет прироста

# def gain(left_labels, right_labels, root_gini):

#     # доля выборки, ушедшая в левое поддерево
#     p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
#     return root_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

# # Разбиение датасета в узле

# def split(data, labels, column_index, t):
    
#     left = np.where(data[:, column_index] <= t)
#     right = np.where(data[:, column_index] > t)
        
#     true_data = data[left]
#     false_data = data[right]
    
#     true_labels = labels[left]
#     false_labels = labels[right]
        
#     return true_data, false_data, true_labels, false_labels

# # Нахождение наилучшего разбиения

# def find_best_split(data, labels):
    
#     #  обозначим минимальное количество объектов в узле
#     min_samples_leaf = 5

#     root_gini = gini(labels)

#     best_gain = 0
#     best_t = None
#     best_index = None
    
#     n_features = data.shape[1]
    
#     for index in range(n_features):
#         # будем проверять только уникальные значения признака, исключая повторения
#         t_values = np.unique(data[:, index])
        
#         for t in t_values:
#             true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
#             #  пропускаем разбиения, в которых в узле остается менее 5 объектов
#             if len(true_data) < min_samples_leaf or len(false_data) < min_samples_leaf:
#                 continue
            
#             current_gain = gain(true_labels, false_labels, root_gini)
            
#             #  выбираем порог, на котором получается максимальный прирост качества
#             if current_gain > best_gain:
#                 best_gain, best_t, best_index = current_gain, t, index

#     return best_gain, best_t, best_index

# # Построение дерева с помощью рекурсивной функции

# def build_tree(data, labels):
#     gain, t, index = find_best_split(data, labels)

#     #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
#     if gain == 0:
#         return Leaf(data, labels)

#     true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
    
#     # Рекурсивно строим два поддерева
#     true_branch = build_tree(true_data, true_labels)
#     import time
#     print(time.time(), true_branch)
#     false_branch = build_tree(false_data, false_labels)
    
#     print(time.time(), false_branch)
    
#     # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
#     return Node(index, t, true_branch, false_branch)

# def classify_object(obj, node):

#     #  Останавливаем рекурсию, если достигли листа
#     if isinstance(node, Leaf):
#         answer = node.prediction
#         return answer

#     if obj[node.index] <= node.t:
#         return classify_object(obj, node.true_branch)
#     else:
#         return classify_object(obj, node.false_branch)

# def predict(data, tree):
    
#     classes = []
#     for obj in data:
#         prediction = classify_object(obj, tree)
#         classes.append(prediction)
#     return classes

# # Введем функцию подсчета точности как доли правильных ответов
# def accuracy_metric(actual, predicted):
#     correct = 0
#     for i in range(len(actual)):
#         if actual[i] == predicted[i]:
#             correct += 1
#     return correct / float(len(actual)) * 100.0

In [4]:
# train_data, test_data, train_labels, test_labels = train_test_split(train_new[X].values, 
#                                                                     np.squeeze(train_new[Y].values),
#                                                                     test_size = 0.3,
#                                                                     random_state = 1)

In [5]:
# my_tree = build_tree(train_data, train_labels)

In [6]:
# train_answers = predict(train_data, my_tree)
# test_answers = predict(test_data, my_tree)

In [7]:
# train_accuracy = accuracy_metric(train_labels, train_answers)
# test_accuracy = accuracy_metric(test_labels, test_answers)

# print(f'Train accuracy', train_accuracy)
# print(f'Test accuracy', test_accuracy)

In [8]:
# Случайное дерево 0.72

np.random.seed(42)

def get_bootstrap(data, labels, N):
    n_samples = data.shape[0] # размер совпадает с исходной выборкой
    bootstrap = []
    
    for i in range(N):
        
        sample_index = np.random.randint(0, n_samples, size=n_samples)
        b_data = data[sample_index]
        b_labels = labels[sample_index]
        
        bootstrap.append((b_data, b_labels))
        
    return bootstrap

def get_subsample(len_sample):
    # будем сохранять не сами признаки, а их индексы
    sample_indexes = list(range(len_sample))

    len_subsample = int(np.sqrt(len_sample))
    
    subsample = np.random.choice(sample_indexes, size=len_subsample, replace=False)

    return subsample

# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
            
        # найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction        

# Расчет критерия Джини

def gini(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity

# Расчет прироста

def gain(left_labels, right_labels, root_gini):

    # доля выборки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return root_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

# Разбиение датасета в узле

def split(data, labels, column_index, t):
    
    left = np.where(data[:, column_index] <= t)
    right = np.where(data[:, column_index] > t)
        
    true_data = data[left]
    false_data = data[right]
    
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

# Нахождение наилучшего разбиения

def find_best_split(data, labels):
    
    #  обозначим минимальное количество объектов в узле
    min_leaf_samples = 5

    root_gini = gini(labels)

    best_gain = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    feature_subsample_indices = get_subsample(n_features) # выбираем случайные признаки
    
    for index in feature_subsample_indices:
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique(data[:, index])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
#             if len(true_data) < min_leaf_samples or len(false_data) < min_leaf_samples:
#                 continue
            
            current_gain = gain(true_labels, false_labels, root_gini)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_gain > best_gain:
                best_gain, best_t, best_index = current_gain, t, index

    return best_gain, best_t, best_index

# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels):

    gain, t, index = find_best_split(data, labels)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if gain == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels)
    false_branch = build_tree(false_data, false_labels)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

def random_forest(data, labels, n_trees):
    forest = []
    bootstrap = get_bootstrap(data, labels, n_trees)
    
    for b_data, b_labels in bootstrap:
        forest.append(build_tree(b_data, b_labels))
        
    return forest

# Функция классификации отдельного объекта

def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

# Функция классификации отдельного объекта

def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

# предсказание голосованием деревьев

def tree_vote(forest, data):

    # добавим предсказания всех деревьев в список
    predictions = []
    for tree in forest:
        predictions.append(predict(data, tree))
#     print(predictions)

    # сформируем список с предсказаниями для каждого объекта
    predictions_per_object = list(zip(*predictions))
#     print(predictions_per_object)

    # выберем в качестве итогового предсказания для каждого объекта то,
    # за которое проголосовало большинство деревьев
    voted_predictions = []
    for obj in predictions_per_object:
        voted_predictions.append(max(set(obj), key=obj.count))
        
    return voted_predictions

# Введем функцию подсчета точности как доли правильных ответов

def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

def standard_scale(x):
    res = (x - x.mean(axis=0)) / x.std(axis=0)
    return res

In [9]:
train_data, test_data, train_labels, test_labels = train_test_split(train_new[X].values, 
                                                                    np.squeeze(train_new[Y].values),
                                                                    test_size = 0.3,
                                                                    random_state = 1)

In [10]:
n_trees = 9
my_forest_50 = random_forest(train_data, train_labels, n_trees)

# Получим ответы для обучающей выборки 
train_answers = tree_vote(my_forest_50, train_data)
# И получим ответы для тестовой выборки
test_answers = tree_vote(my_forest_50, test_data)

# Точность на обучающей выборке
train_accuracy = accuracy_metric(train_labels, train_answers)
print(f'Точность случайного леса из {n_trees} деревьев на обучающей выборке: {train_accuracy:.3f}')

# Точность на тестовой выборке
test_accuracy = accuracy_metric(test_labels, test_answers)
print(f'Точность случайного леса из {n_trees} деревьев на тестовой выборке: {test_accuracy:.3f}')

Точность случайного леса из 9 деревьев на обучающей выборке: 94.286
Точность случайного леса из 9 деревьев на тестовой выборке: 89.400


In [11]:
# predictions = predict(test_new.values, my_tree)
predictions = tree_vote(my_forest_50, test_new.values)
submit = sample_submission
submit.head()

,Id,choose
0,10000,0.5
1,10001,0.5
2,10002,0.5
3,10003,0.5
4,10004,0.5


In [12]:
submit['choose'] = predictions
submit.head()

,Id,choose
0,10000,0
1,10001,0
2,10002,0
3,10003,0
4,10004,0


In [13]:
submit.to_csv('submit.csv', index=False)